In [4]:
import sys, string, os
from os.path import abspath, dirname, join
import h5py
import numpy as np
import matplotlib.pyplot as plt
from os.path import abspath, dirname, join
import ac_analysis

In [5]:
from ac_analysis.model.annotations import load_from_h5

In [11]:
#DATA_PATH = join(dirname(abspath(ac_analysis.__file__)), 'tests', 'data')

path_name = '/home/genia/rigdata/wobbuffet/170109_ENG-SYS_02_wobbuffet_WAV13R07C12_cycle1/P_00_170110023618_ggc3-keeper_ac-analysis_v11.19.1'

f = h5py.File(path_name + '/annotations.h5')

In [12]:
f.keys()

[u'cells', u'description', u'experiments', u'metadata', u'version']

In [15]:
f['cells'].value

array(['b00c0000', 'b00c0001', 'b00c0002', ..., 'b15c8189', 'b15c8190',
       'b15c8191'], 
      dtype='|S8')

In [8]:
DATA_PATH = join(dirname(abspath(ac_analysis.__file__)), 'tests', 'data')

In [18]:
from ac_analysis.model.annotations import load_from_h5

#make annotation object to get rep based annotations 
#anno_path = "/net/connor/170124_SIG-A_03_blastoise_WAV06R05C15/P_00_170124230837_ggc3-keeper_ac-analysis_v11.19.1/annotations.h5"
anno_path = "/home/genia/rigdata/wobbuffet/170109_ENG-SYS_02_wobbuffet_WAV13R07C12_cycle1/P_00_170110023618_ggc3-keeper_ac-analysis_v11.19.1/annotations.h5"
annos = load_from_h5(anno_path)
cells = annos.cells  #grab all cells; same as f['cells'].value using h5py?
neg_low, units = annos.get_rep_annotation('neg_low') #grab neg_low matrix shape = (cells, reps)

In [19]:
neg_low.shape  #single rep of neg_low

(131072, 1)

In [20]:
cells

array(['b00c0000', 'b00c0001', 'b00c0002', ..., 'b15c8189', 'b15c8190',
       'b15c8191'], 
      dtype='|S8')

In [22]:
mask = np.all(np.isnan(neg_low) | np.equal(neg_low, 0), axis=1) #remove rows with only nan and zero (probably dont want to just remove zeros)
filt_neg_low = neg_low[~mask]   #only get cells where neg_low is not all nan or all zero
filt_cells = cells[~mask]  #get cell names based on neg_low mask 

# neg_low_cell = neg_low[cells == 'b00c0810']
# neg_low_cell

In [23]:
mask

array([ True,  True,  True, ...,  True,  True,  True], dtype=bool)

In [14]:
filt_neg_low

array([[-15.82650375],
       [-14.90199852],
       [-29.95208168],
       ..., 
       [-15.62517452],
       [-17.76834297],
       [-44.0656929 ]], dtype=float32)

In [15]:
filt_cells

array(['b00c0029', 'b00c0359', 'b00c0413', ..., 'b15c3338', 'b15c4868',
       'b15c5003'], 
      dtype='|S8')

In [24]:
#now build 
[name for name in annos.get_cell_annotation_names() if 'zero' in name]  #don't need but useful for checking annotation names

[u'vmzero_end', u'oc_calibration_cmzero', u'oc_calibration_vmzero']

In [33]:
vmzeros, unit = annos.get_cell_annotation('oc_calibration_vmzero')
vmzeros_end, unit = annos.get_cell_annotation('vmzero_end') #returns numpy matrix, and string (matrix, string)
filt_vmzeros = vmzeros[~mask]
plt.hist(filt_vmzeros, bins=10); plt.show()  #plot to show vmzeor distribution

#now what you have is neg_low minus vmzero
flat_filt_neg_low = np.ravel(filt_neg_low)  #flatten matrix to array to do subtraction of vmzero 
print(flat_filt_neg_low.shape, filt_vmzeros.shape) 
neg_low_minus_vmzero = flat_filt_neg_low - filt_vmzeros  #subtract vmzero
neg_low_minus_vmzero.shape  #confirm proper shape 


final_mask = [neg_low_minus_vmzero < 5]
filt_cells[final_mask]



((8700,), (8700,))


array(['b00c0029', 'b00c0359', 'b00c0413', ..., 'b15c3338', 'b15c4868',
       'b15c5003'], 
      dtype='|S8')

In [31]:
vmzero_minus_neg_low = filt_vmzeros - flat_filt_neg_low
final_mask = [vmzero_minus_neg_low > 0]
vmzero_minus_neg_low.max()

cell_vmzero = filt_vmzeros[filt_cells == 'b00c0810']

#### Look at a particula cell found on ac-dvt (with the problem of neg_low > vmzero)

In [35]:
neg_low_cell = neg_low[cells == 'b00c0810']
neg_low_cell

# Did not seem to find?

array([[ nan]], dtype=float32)